# Jupyter notebook based on ImageD11 to process scanning 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 26/02/2024

In [ ]:
# There is a bug with the current version of ImageD11 in the site-wide Jupyter env.
# This has been fixed here: https://github.com/FABLE-3DXRD/ImageD11/commit/4af88b886b1775585e868f2339a0eb975401468f
# Until a new release has been made and added to the env, we need to get the latest version of ImageD11 from GitHub
# Put it in your home directory
# USER: Change the path below to point to your local copy of ImageD11:

import os

username = os.environ.get("USER")

id11_code_path = f"/home/esrf/{username}/Code/ImageD11"

import sys

sys.path.insert(0, id11_code_path)

In [ ]:
# import functions we need

import concurrent.futures
import timeit
import glob
import pprint
from shutil import rmtree
import time
from functools import partial

import matplotlib
%matplotlib ipympl

from scipy.optimize import curve_fit

import h5py
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

import ImageD11.columnfile
from ImageD11.sinograms import properties, roi_iradon
from ImageD11.blobcorrector import eiger_spatial
from ImageD11.grain import grain

from skimage.filters import threshold_otsu
from skimage.morphology import convex_hull_image

import ImageD11.nbGui.nb_utils as utils

import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
# define our functions

def read_grains(ds):
    with h5py.File(ds.grainsfile, 'r') as hin:      
        grains_group = 'grains'
        
        grains = []
        for gid_string in tqdm(sorted(hin[grains_group].keys(), key=lambda x: int(x))):
            gg = hin[grains_group][gid_string]
            ubi = gg.attrs['ubi'][:]
            g = ImageD11.grain.grain(ubi)
            g.gid = int(gid_string)
            grains.append(g)
    
    return grains


def map_grain_from_peaks(g, flt, ds):
    """
    Computes sinogram
    flt is already the peaks for this grain
    Returns angles, sino
    """   
    NY = len(ds.ybincens)  # number of y translations
    iy = np.round((flt.dty - ds.ybincens[0]) / (ds.ybincens[1]-ds.ybincens[0])).astype(int)  # flt column for y translation index

    # The problem is to assign each spot to a place in the sinogram
    hklmin = g.hkl_2d_strong.min(axis=1)  # Get minimum integer hkl (e.g -10, -9, -10)
    dh = g.hkl_2d_strong - hklmin[:,np.newaxis]  # subtract minimum hkl from all integer hkls
    de = (g.etasigns_2d_strong.astype(int) + 1)//2  # something signs related
    #   4D array of h,k,l,+/-
    # pkmsk is whether a peak has been observed with this HKL or not
    pkmsk = np.zeros(list(dh.max(axis=1) + 1 )+[2,], int)  # make zeros-array the size of (max dh +1) and add another axis of length 2
    pkmsk[ dh[0], dh[1], dh[2], de ] = 1  # we found these HKLs for this grain
    #   sinogram row to hit
    pkrow = np.cumsum(pkmsk.ravel()).reshape(pkmsk.shape) - 1  #
    # counting where we hit an HKL position with a found peak
    # e.g (-10, -9, -10) didn't get hit, but the next one did, so increment

    npks = pkmsk.sum( )
    destRow = pkrow[ dh[0], dh[1], dh[2], de ] 
    sino = np.zeros( ( npks, NY ), 'f' )
    hits = np.zeros( ( npks, NY ), 'f' )
    angs = np.zeros( ( npks, NY ), 'f' )
    adr = destRow * NY + iy 
    # Just accumulate 
    sig = flt.sum_intensity
    ImageD11.cImageD11.put_incr64( sino, adr, sig )
    ImageD11.cImageD11.put_incr64( hits, adr, np.ones(len(de),dtype='f'))
    ImageD11.cImageD11.put_incr64( angs, adr, flt.omega)
    
    sinoangles = angs.sum( axis = 1) / hits.sum( axis = 1 )
    # Normalise:
    sino = (sino.T/sino.max( axis=1 )).T
    # Sort (cosmetic):
    order = np.lexsort((np.arange(npks), sinoangles))
    sinoangles = sinoangles[order]
    ssino = sino[order].T
    return sinoangles, ssino, hits[order].T

def do_sinos(g, hkltol=0.25):
    flt = utils.tocolf({p:p2d[p][g.peaks_2d] for p in p2d}, par_path, dxfile=e2dx_path, dyfile=e2dy_path)  # convert it to a columnfile and spatially correct
    
    hkl_real = np.dot(g.ubi, (flt.gx, flt.gy, flt.gz))  # calculate hkl of all assigned peaks
    hkl_int = np.round(hkl_real).astype(int) # round to nearest integer
    dh = ((hkl_real - hkl_int)**2).sum(axis = 0)  # calculate square of difference

    # g.dherrall = dh.mean()  # mean hkl error across all assigned peaks
    # g.npksall = flt.nrows  # total number of assigned peaks
    flt.filter(dh < hkltol*hkltol)  # filter all assigned peaks to be less than hkltol squared
    hkl_real = np.dot(g.ubi, (flt.gx, flt.gy, flt.gz))  # recalculate error after filtration
    hkl_int = np.round(hkl_real).astype(int)
    dh = ((hkl_real - hkl_int)**2).sum(axis = 0)
    # g.dherr = dh.mean()  # dherr is mean hkl error across assigned peaks after hkltol filtering
    # g.npks = flt.nrows  # total number of assigned peaks after hkltol filtering
    g.etasigns_2d_strong = np.sign(flt.eta)
    g.hkl_2d_strong = hkl_int  # integer hkl of assigned peaks after hkltol filtering
    g.sinoangles, g.ssino, g.hits = map_grain_from_peaks(g, flt, ds)
    return i,g


def run_iradon_id11(grain, pad=20, y0=0, workers=1, sample_mask=None, apply_halfmask=False, mask_central_zingers=False):
    outsize = grain.ssino.shape[0] + pad
    
    if apply_halfmask:
        halfmask = np.zeros_like(grain.ssino)

        halfmask[:len(halfmask)//2-1, :] = 1
        halfmask[len(halfmask)//2-1, :] = 0.5
        
        ssino_to_recon = grain.ssino * halfmask
    else:
        ssino_to_recon = grain.ssino
        
    # # pad the sample mask
    # sample_mask_padded = np.pad(sample_mask, pad//2)

    
    # Perform iradon transform of grain sinogram, store result (reconstructed grain shape) in g.recon
    grain.recon = ImageD11.sinograms.roi_iradon.iradon(ssino_to_recon, 
                                                       theta=grain.sinoangles, 
                                                       mask=sample_mask,
                                                       output_size=outsize,
                                                       projection_shifts=np.full(grain.ssino.shape, -y0),
                                                       filter_name='hamming',
                                                       interpolation='linear',
                                                       workers=workers)
    
    if mask_central_zingers:
        grs = grain.recon.shape[0]
        xpr, ypr = -grs//2 + np.mgrid[:grs, :grs]
        inner_mask_radius = 25
        outer_mask_radius = inner_mask_radius + 2

        inner_circle_mask = (xpr ** 2 + ypr ** 2) < inner_mask_radius ** 2
        outer_circle_mask = (xpr ** 2 + ypr ** 2) < outer_mask_radius ** 2

        mask_ring = inner_circle_mask & outer_circle_mask
        # we now have a mask to apply
        fill_value = np.median(grain.recon[mask_ring])
        grain.recon[inner_circle_mask] = fill_value
    
    return grain

cmp = {'compression':'gzip',
       'compression_opts': 2,
       'shuffle' : True }

def save_array(grp, name, ary):
    hds = grp.require_dataset(name, 
                              shape=ary.shape,
                              dtype=ary.dtype,
                              **cmp)
    hds[:] = ary
    return hds

def save_grains_for_mlem(grains, ds, y0):
    with h5py.File(ds.grainsfile, 'r+') as hout:
        try:
            grp = hout.create_group('peak_assignments')
        except ValueError:
            grp = hout['peak_assignments']

        ds_gord = save_array( grp, 'gord', gord )
        ds_gord.attrs['description'] = 'Grain ordering: g[i].pks = gord[ inds[i] : inds[i+1] ]'
        ds_inds = save_array( grp, 'inds', inds )
        ds_inds.attrs['description'] = 'Grain indices: g[i].pks = gord[ inds[i] : inds[i+1] ]'
        
        grains_group = 'grains'
        for g in tqdm(grains):
            gg = hout[grains_group][str(g.gid)]
            # save stuff for sinograms
            
            save_array(gg, 'ssino', g.ssino).attrs['description'] = 'Sinogram of peak intensities sorted by omega'
            save_array(gg, 'sinoangles', g.sinoangles).attrs['description'] = 'Projection angles for sinogram'
            save_array(gg, 'og_recon', g.og_recon).attrs['description'] = 'Original ID11 iRadon reconstruction'
            save_array(gg, 'circle_mask', whole_sample_mask).attrs['description'] = 'Reconstruction mask to use for MLEM'
            
            # might as well save peaks stuff while we're here
            save_array(gg, 'translation', g.translation).attrs['description'] = 'Grain translation in lab frame'
            save_array(gg, 'peaks_2d_sinograms', g.peaks_2d).attrs['description'] = "2D peaks from strong 4D peaks that were assigned to this grain for sinograms"
            save_array(gg, 'peaks_4d_sinograms', g.peaks_4d).attrs['description'] = "Strong 4D peaks that were assigned to this grain for sinograms"

            gg.attrs['cen'] = g.cen
            gg.attrs['y0'] = y0
            
            
def prepare_mlem_bash(ds, grains, pad, is_half_scan, n_simultaneous_jobs=50, cores_per_task=8, niter=50):
    
    slurm_mlem_path = os.path.join(ds.analysispath, "slurm_mlem")

    if os.path.exists(slurm_mlem_path):
        print(f"Removing {slurm_mlem_path}")
        rmtree(slurm_mlem_path)

    os.mkdir(slurm_mlem_path)
    
    recons_path = os.path.join(ds.analysispath, "mlem_recons")

    if os.path.exists(recons_path):
        print(f"Removing {recons_path}")
        rmtree(recons_path)

    os.mkdir(recons_path)
    
    if is_half_scan:
        dohm = "Yes"
        mask_cen = "Yes"
    else:
        dohm = "No"
        mask_cen = "No"
    
    bash_script_path = os.path.join(slurm_mlem_path, ds.dsname + '_mlem_recon_slurm.sh')
    python_script_path = os.path.join(id11_code_path, "ImageD11/nbGui/S3DXRD/run_mlem_recon.py") 
    outfile_path =  os.path.join(slurm_mlem_path, ds.dsname + '_mlem_recon_slurm_%A_%a.out')
    errfile_path =  os.path.join(slurm_mlem_path, ds.dsname + '_mlem_recon_slurm_%A_%a.err')
    log_path = os.path.join(slurm_mlem_path, ds.dsname + '_mlem_recon_slurm_$SLURM_ARRAY_JOB_ID_$SLURM_ARRAY_TASK_ID.log')

    reconfile = os.path.join(recons_path, ds.dsname + "_mlem_recon_$SLURM_ARRAY_TASK_ID.txt")

    bash_script_string = f"""#!/bin/bash
#SBATCH --job-name=mlem-recon
#SBATCH --output={outfile_path}
#SBATCH --error={errfile_path}
#SBATCH --array=0-{len(grains)-1}%{n_simultaneous_jobs}
#SBATCH --time=02:00:00
# define memory needs and number of tasks for each array job
#SBATCH --ntasks=1
#SBATCH --cpus-per-task={cores_per_task}
#
date
echo python3 {python_script_path} {ds.grainsfile} $SLURM_ARRAY_TASK_ID {reconfile} {pad} {niter} {dohm} {mask_cen} > {log_path} 2>&1
python3 {python_script_path} {ds.grainsfile} $SLURM_ARRAY_TASK_ID {reconfile} {pad} {niter} {dohm} {mask_cen} > {log_path} 2>&1
date
    """
    
    with open(bash_script_path, "w") as bashscriptfile:
        bashscriptfile.writelines(bash_script_string)
        
    return bash_script_path, recons_path


def save_grains(grains, ds):
    with h5py.File(ds.grainsfile, 'r+') as hout:
        try:
            grp = hout.create_group('slice_recon')
        except ValueError:
            grp = hout['slice_recon']
        save_array(grp, 'intensity', raw_intensity_array).attrs['description'] = 'Raw intensity array for all grains'
        save_array(grp, 'labels', grain_labels_array).attrs['description'] = 'Grain labels array for all grains'
        
        ipfxdset = save_array(grp, 'ipf_x_col_map', rgb_x_array)
        ipfxdset.attrs['description'] = 'IPF X color at each pixel'
        ipfxdset.attrs['CLASS'] = 'IMAGE'
        ipfydset = save_array(grp, 'ipf_y_col_map', rgb_y_array)
        ipfydset.attrs['description'] = 'IPF Y color at each pixel'
        ipfydset.attrs['CLASS'] = 'IMAGE'
        ipfzdset = save_array(grp, 'ipf_z_col_map', rgb_z_array)
        ipfzdset.attrs['description'] = 'IPF Z color at each pixel'
        ipfzdset.attrs['CLASS'] = 'IMAGE'
        
        grains_group = 'grains'

        for g in tqdm(grains):
            gg = hout[grains_group][str(g.gid)]
            
            save_array(gg, 'recon', g.recon).attrs['description'] = 'Final reconstruction'
            
            
# without a mask, MLEM can introduce artifacts in the corners
# so we can manually mask those out

# we can incoporate our own mask too
# by modifying the below function

def apply_manual_mask(mask_in):
    mask_out = mask_in.copy()
    
    # mask_out[200:, 250:] = 0
    
    return mask_out

In [ ]:
# NOTE: For old datasets before the new directory layout structure, we don't distinguish between RAW_DATA and PROCESSED_DATA

### USER: specify your experimental directory

rawdata_path = "/home/esrf/james1997a/Data/ihma439/id11/20231211/RAW_DATA"

!ls -lrt {rawdata_path}

### USER: specify where you want your processed data to go

processed_data_root_dir = "/home/esrf/james1997a/Data/ihma439/id11/20231211/PROCESSED_DATA/James/20240226"

In [ ]:
# USER: pick a sample and a dataset you want to segment

sample = "FeAu_0p5_tR_nscope"
dataset = "top_250um"

In [ ]:
# desination of H5 files

dset_path = os.path.join(processed_data_root_dir, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")

par_path = os.path.join(processed_data_root_dir, 'Fe_refined.par')

e2dx_path = os.path.join(processed_data_root_dir, '../../CeO2/e2dx_E-08-0173_20231127.edf')
e2dy_path = os.path.join(processed_data_root_dir, '../../CeO2/e2dy_E-08-0173_20231127.edf')

In [ ]:
# Load the dataset (for motor positions, not sure why these are not in peaks)
ds = ImageD11.sinograms.dataset.load(dset_path)

In [ ]:
# Import 4D peaks

cf_4d = ImageD11.columnfile.columnfile(ds.col4dfile)

cf_4d.parameters.loadparameters(par_path)
cf_4d.updateGeometry()

print(f"Read {cf_4d.nrows} 4D peaks")

In [ ]:
grains = read_grains(ds)

for grain in grains:
    # print(grain.gid)
    grain.a = np.cbrt(np.linalg.det(grain.ubi))
    
print(f"{len(grains)} grains imported")

In [ ]:
# here we are filtering our peaks (cf_4d) to select only the strongest ones
# this time as opposed to indexing, our frac is slightly weaker but we are NOT filtering in dstar!!!!!
# this means many more peaks per grain = stronger sinograms

# USER: modify the "frac" parameter below and re-run the cell until the orange dot sits nicely on the "elbow" of the blue line
# this indicates the fractional intensity cutoff we will select
# if the blue line does not look elbow-shaped in the logscale plot, try changing the "doplot" parameter (the y scale of the logscale plot) until it does

cf_strong = utils.selectpeaks(cf_4d, frac=0.995, dsmax=cf_4d.ds.max(), doplot=0.9)
print(cf_4d.nrows)
cf_strong.nrows

In [ ]:
# # now let's do a whole-sample tomographic reconstruction

In [ ]:
# If the sinograms are only half-sinograms (we scanned dty across half the sample rather than the full sample), set the below to true:
is_half_scan = False

In [ ]:
if is_half_scan:
    utils.correct_half_scan(ds)

In [ ]:
utils.assign_peaks_to_grains(grains, cf_strong, tol=0.25)

print("Storing peak data in grains")
# iterate through all the grains
for g in tqdm(grains):
    # store this grain's peak indices so we know which 4D peaks we used for indexing
    g.mask_4d = cf_strong.grain_id == g.gid
    g.peaks_4d = cf_strong.index[cf_strong.grain_id == g.gid]

In [ ]:
for grain in grains:
    # grain.peaks_4d_selected, grain.cen, grain.dx, grain.dy = utils.graincen(grain.gid, cf_strong, doplot=True, nsigma=1)
    grain.rgb_z = utils.grain_to_rgb(grain, ax=(0,0,1),)# symmetry = Symmetry.cubic)
    grain.rgb_y = utils.grain_to_rgb(grain, ax=(0,1,0),)# symmetry = Symmetry.cubic)
    grain.rgb_x = utils.grain_to_rgb(grain, ax=(1,0,0),)# symmetry = Symmetry.cubic)
    utils.fit_grain_position_from_sino(grain, cf_strong)

In [ ]:
# make sure we get cen right (centre of rotation should be the middle of dty)
fig, ax = plt.subplots()
ax.plot([g.cen for g in grains])

plt.show()

In [ ]:
c0 = np.median([g.cen for g in grains])

print('Center of rotation in dty', c0)

y0 = c0/2

print('y0 is', y0)

In [ ]:
# generate sinogram for whole sample

whole_sample_sino, xedges, yedges = np.histogram2d(cf_4d.dty, cf_4d.omega, bins=[ds.ybinedges, ds.obinedges])

fig, ax = plt.subplots()
ax.imshow(whole_sample_sino, interpolation="nearest", vmin=0)
ax.set_aspect(4)
plt.show()

In [ ]:
# "quick" whole-sample reconstruction

pad = 50

outsize = whole_sample_sino.shape[0] + pad

nthreads = len(os.sched_getaffinity(os.getpid()))

if is_half_scan:
    halfmask = np.zeros_like(whole_sample_sino)

    halfmask[:len(halfmask)//2-1, :] = 1
    halfmask[len(halfmask)//2-1, :] = 0.5

    ssino_to_recon = whole_sample_sino * halfmask
else:
    ssino_to_recon = whole_sample_sino

recon = ImageD11.sinograms.roi_iradon.iradon(ssino_to_recon, 
                                           theta=ds.obincens, 
                                           output_size=outsize,
                                           projection_shifts=np.full(whole_sample_sino.shape, -y0),
                                           filter_name='hamming',
                                           interpolation='linear',
                                           workers=nthreads)

In [ ]:
# we should be able to easily segment this using scikit-image
recon_man_mask = apply_manual_mask(recon)

thresh = threshold_otsu(recon_man_mask)

# we can also override the threshold if we don't like it:

# thresh = 0.05

binary = recon_man_mask > thresh

chull = convex_hull_image(binary)

fig, axs = plt.subplots(1, 3, sharex=True, sharey=True, constrained_layout=True)
axs[0].imshow(recon_man_mask, vmin=0)
axs[1].imshow(binary)
axs[2].imshow(chull)

axs[0].set_title("Reconstruction")
axs[1].set_title("Binarised threshold")
axs[2].set_title("Convex hull")

plt.show()

In [ ]:
whole_sample_mask = chull

In [ ]:
fig, ax = plt.subplots()
m = cf_strong.grain_id >= 0
ax.scatter(cf_strong.omega[m], cf_strong.dty[m], c=cf_strong.grain_id[m])
plt.show()

In [ ]:
mean_unit_cell_lengths = [grain.a for grain in grains]

fig, ax = plt.subplots()
ax.plot(mean_unit_cell_lengths)
ax.set_xlabel("Grain ID")
ax.set_ylabel("Unit cell length")
plt.show()

a0 = np.median(mean_unit_cell_lengths)
    
print(a0)

In [ ]:
utils.plot_grain_sinograms(grains, cf_strong)

In [ ]:
np.sum(grains[0].mask_4d)

In [ ]:
# plt.style.use('dark_background')
fig, ax = plt.subplots(2,2, figsize=(12,12))
a = ax.ravel()
x = [g.dx for g in grains]
y = [g.dy for g in grains]
s = [g.mask_4d.sum()/10 for g in grains]
a[0].scatter(x, y, c=[g.rgb_z for g in grains], s=s)
a[0].set(title='IPF color Z',  aspect='equal')
a[1].scatter(x, y, c=[g.rgb_y for g in grains], s=s)
a[1].set(title='IPF color Y', aspect='equal')
a[2].scatter(x, y, c=[g.rgb_x for g in grains], s=s)
a[2].set(title='IPF color X',  aspect='equal')
a[3].scatter(x, y, c=s)
a[3].set(title='Number of 4D peaks', aspect='equal')

fig.supxlabel("Lab x")
fig.supylabel("Lab y")

plt.show()

In [ ]:
# populate translations of grains
for g in grains:
    g.translation = np.array([g.dx, g.dy, 0])

In [ ]:
# Big scary block
# Must understand what this does!

# Ensure cf is sorted by spot3d_id
# NOTE: spot3d_id should be spot4d_id, because we have merged into 4D?
assert (np.argsort(cf_strong.spot3d_id) == np.arange(cf_strong.nrows)).all()

# load the 2d peak labelling output
pks = ImageD11.sinograms.properties.pks_table.load(ds.pksfile)

# Grab the 2d peak centroids
p2d = pks.pk2d(ds.omega, ds.dty)

# NOTE: These are not spatially corrected?!

numba_order, numba_histo = utils.counting_sort(p2d['spot3d_id'])

grain_2d_id = utils.palloc(p2d['spot3d_id'].shape, np.dtype(int))

cleanid = cf_strong.grain_id.copy()

utils.find_grain_id(cf_strong.spot3d_id, cleanid, p2d['spot3d_id'], grain_2d_id, numba_order)

gord, counts = utils.counting_sort(grain_2d_id)

inds = np.concatenate(((0,), np.cumsum(counts)))

# I think what we end up with is:
# inds
# this is an array which tells you which 2D spots each grain owns
# the 2D spots are sorted by spot ID
# inds tells you for each grain were you can find its associated 2D spots

In [ ]:
# now our 2D peak assignments are known, let's populate our grain objects with our 2D peaks

for grain in tqdm(grains):
    i = grain.gid
    grain.peaks_2d = gord[inds[i+1] : inds[i+2]]
    # grain.mask_2d = np.isin(cf_2d.index, grain.peaks_2d)

In [ ]:
# Determine sinograms of all grains

nthreads = len(os.sched_getaffinity(os.getpid()))

with concurrent.futures.ThreadPoolExecutor(max_workers= max(1,nthreads-1)) as pool:
    for i in tqdm(pool.map(do_sinos, grains), total=len(grains)):
        pass

In [ ]:
# Show sinogram of single grain

g = grains[0]

fig, ax = plt.subplots()

ax.imshow((g.ssino/g.ssino.mean(axis=0)), norm=matplotlib.colors.LogNorm(), interpolation='nearest', origin="lower")

plt.show()

In [ ]:
# if you want, you can override the y0 value here

# y0 = 1.5  # for example!

y0 = c0/2

In [ ]:
g = grains[1]

run_iradon_id11(g, pad=pad, y0=y0, workers=max(nthreads, 20), sample_mask=whole_sample_mask, apply_halfmask=is_half_scan, mask_central_zingers=is_half_scan)

In [ ]:
g = grains[1]

fig, axs = plt.subplots(1,2, figsize=(10,5))
axs[0].imshow(g.recon, vmin=0)
axs[0].set_title("ID11 iradon")
axs[1].imshow(g.ssino, aspect='auto')
axs[1].set_title("ssino")

plt.show()

In [ ]:
nthreads = len(os.sched_getaffinity(os.getpid()))

run_this_iradon = partial(run_iradon_id11, pad=pad, y0=y0, sample_mask=whole_sample_mask, workers=1, apply_halfmask=is_half_scan, mask_central_zingers=is_half_scan)

with concurrent.futures.ThreadPoolExecutor( max_workers= max(1,nthreads-1) ) as pool:
    for i in tqdm(pool.map(run_this_iradon, grains), total=len(grains)):
        pass

In [ ]:
for grain in grains:
    grain.og_recon = grain.recon

In [ ]:
fig, a = plt.subplots(1,2,figsize=(10,5))
rec = a[0].imshow(grains[8].og_recon, vmin=0)
sin = a[1].imshow(grains[8].ssino, aspect='auto')

# Function to update the displayed image based on the selected frame
def update_frame(i):
    rec.set_array(grains[i].og_recon)
    sin.set_array(grains[i].ssino)
    a[0].set(title=str(i))
    fig.canvas.draw()

# Create a slider widget to select the frame number
frame_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(grains) - 1,
    step=1,
    description='Grain:'
)

interact(update_frame, i=frame_slider)

plt.show()

In [ ]:
f,a = plt.subplots( 1,3, figsize=(15,5) )
ty, tx = utils.triangle().T
for i,title in enumerate( 'xyz' ):
    ax = np.zeros(3)
    ax[i] = 1.
    hkl = [utils.crystal_direction_cubic( g.ubi, ax ) for g in grains]
    xy = np.array([utils.hkl_to_pf_cubic(h) for h in hkl ])
    rgb = np.array([utils.hkl_to_color_cubic(h) for h in hkl ])
    for j in range(len(grains)):
        grains[j].rgb = rgb[j]
    a[i].scatter( xy[:,1], xy[:,0], c = rgb )   # Note the "x" axis of the plot is the 'k' direction and 'y' is h (smaller)
    a[i].set(title=title, aspect='equal', facecolor='silver', xticks=[], yticks=[])
    a[i].plot( tx, ty, 'k-', lw = 1 )

In [ ]:
rgb_x_array, rgb_y_array, rgb_z_array, grain_labels_array, raw_intensity_array = utils.build_slice_arrays(grains, cutoff_level=0.4)

In [ ]:
# plot initial output

fig, ax = plt.subplots(constrained_layout=True)
ax.imshow(rgb_z_array)
plt.show()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
ax.imshow(grain_labels_array)  # originally 1,2,0
ax.set_title("Grain label map")
plt.show()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
ax.imshow(raw_intensity_array)
ax.set_title("Raw intensity array")
plt.show()

In [ ]:
# we can clean up these reconstructions using an MLEM iterative recon
# we can use the whole sample shape mask for this

In [ ]:
save_grains_for_mlem(grains, ds, y0=y0)

In [ ]:
bash_script_path, recons_path = prepare_mlem_bash(ds, grains, pad, is_half_scan, n_simultaneous_jobs=50, cores_per_task=8, niter=50)

In [ ]:
utils.slurm_submit_and_wait(bash_script_path, 30)

In [ ]:
# collect results into grain attributes
# the filenames are element position not gid

for i, grain in enumerate(tqdm(grains)):
    grain.recon = np.loadtxt(os.path.join(recons_path, ds.dsname + f"_mlem_recon_{i}.txt"))

In [ ]:
# look at all our grains

n_grains_to_plot = 25

grains_step = len(grains)//n_grains_to_plot

grid_size = np.ceil(np.sqrt(len(grains[::grains_step]))).astype(int)
nrows = (len(grains[::grains_step])+grid_size-1)//grid_size

fig, axs = plt.subplots(grid_size, nrows, figsize=(10,10), layout="constrained", sharex=True, sharey=True)
for i, ax in enumerate(axs.ravel()):
    if i < len(grains[::grains_step]):
    # get corresponding grain for this axis
        g = grains[::grains_step][i]
        ax.imshow(g.recon, vmin=0)
        # ax.invert_yaxis()
        ax.set_title(i)
    
plt.show()

In [ ]:
rgb_x_array, rgb_y_array, rgb_z_array, grain_labels_array, raw_intensity_array = utils.build_slice_arrays(grains, cutoff_level=0.3)

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
ax.imshow(rgb_z_array)
plt.show()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
ax.imshow(raw_intensity_array)
ax.set_title("Sinogram raw intensity map")
plt.show()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
ax.imshow(grain_labels_array)
ax.set_title("Grain label map")
plt.show()

In [ ]:
save_grains(grains, ds)

In [ ]:
if 1:
    raise ValueError("Change the 1 above to 0 to allow 'Run all cells' in the notebook")

In [ ]:
# Now that we're happy with our segmentation parameters, we can run the below cell to do this in bulk for many samples/datasets
# by default this will do all samples in sample_list, all datasets with a prefix of dset_prefix
# you can add samples and datasets to skip

skips_dict = {
    "FeAu_0p5_tR_nscope": ["top_-50um", "top_-100um"]
}

dset_prefix = "top"

sample_list = ["FeAu_0p5_tR_nscope"]
    
samples_dict = utils.find_datasets_to_process(rawdata_path, skips_dict, dset_prefix, sample_list)
    
# manual override:
# samples_dict = {"FeAu_0p5_tR_nscope": ["top_100um", "top_200um"]}
    
# now we have our samples_dict, we can process our data:

par_path = os.path.join(processed_data_root_dir, 'Fe_refined.par')

e2dx_path = os.path.join(processed_data_root_dir, '../../CeO2/e2dx_E-08-0173_20231127.edf')
e2dy_path = os.path.join(processed_data_root_dir, '../../CeO2/e2dy_E-08-0173_20231127.edf')

cf_strong_frac = 0.995
cf_strong_dstol = 0.01

is_half_scan = False

peak_assign_tol = 0.25

manual_threshold = None
# manual_threshold = 0.025

nthreads = len(os.sched_getaffinity(os.getpid()))

pad = 50

# y0 = -1.4

mlem_n_simultaneous_jobs = 50
mlem_cores_per_task = 8
mlem_niter = 50

cutoff_level = 0.2

for sample, datasets in samples_dict.items():
    for dataset in datasets:
        print(f"Processing dataset {dataset} in sample {sample}")
        dset_path = os.path.join(processed_data_root_dir, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")
        if not os.path.exists(dset_path):
            print(f"Missing DataSet file for {dataset} in sample {sample}, skipping")
            continue
        
        print("Importing DataSet object")
        
        ds = ImageD11.sinograms.dataset.load(dset_path)
        print(f"I have a DataSet {ds.dset} in sample {ds.sample}")
        
        if not os.path.exists(ds.grainsfile):
            print(f"Missing grains file for {dataset} in sample {sample}, skipping")
            continue
            
        # check grains file for existance of slice_recon, skip if it's there
        with h5py.File(ds.grainsfile, "r") as hin:
            if "slice_recon" in hin.keys():
                print(f"Already reconstructed {dataset} in {sample}, skipping")
                continue
            
        cf_4d = ImageD11.columnfile.columnfile(ds.col4dfile)
        cf_4d.parameters.loadparameters(par_path)
        cf_4d.updateGeometry()
        
        grains = read_grains(ds)
        
        cf_strong = utils.selectpeaks(cf_4d, frac=cf_strong_frac, dsmax=cf_4d.ds.max(), dstol=cf_strong_dstol)
        
        if is_half_scan:
            utils.correct_half_scan(ds)
            
        utils.assign_peaks_to_grains(grains, cf_strong, tol=peak_assign_tol)
        
        for g in tqdm(grains):
            g.mask_4d = cf_strong.grain_id == g.gid
            g.peaks_4d = cf_strong.index[cf_strong.grain_id == g.gid]
            
        for grain in tqdm(grains):
            # grain.peaks_4d_selected, grain.cen, grain.dx, grain.dy = utils.graincen(grain.gid, cf_strong, doplot=False)
            grain.rgb_z = utils.grain_to_rgb(grain, ax=(0,0,1),)# symmetry = Symmetry.cubic)
            grain.rgb_y = utils.grain_to_rgb(grain, ax=(0,1,0),)# symmetry = Symmetry.cubic)
            grain.rgb_x = utils.grain_to_rgb(grain, ax=(1,0,0),)# symmetry = Symmetry.cubic)
            utils.fit_grain_position_from_sino(grain, cf_strong)
            
        c0 = np.median([g.cen for g in grains])
        
        y0 = c0/2
        
        whole_sample_sino, xedges, yedges = np.histogram2d(cf_4d.dty, cf_4d.omega, bins=[ds.ybinedges, ds.obinedges])
        
        print("Whole sample mask")
        outsize = whole_sample_sino.shape[0] + pad

        if is_half_scan:
            halfmask = np.zeros_like(whole_sample_sino)

            halfmask[:len(halfmask)//2-1, :] = 1
            halfmask[len(halfmask)//2-1, :] = 0.5

            ssino_to_recon = whole_sample_sino * halfmask
        else:
            ssino_to_recon = whole_sample_sino
        
        recon = ImageD11.sinograms.roi_iradon.iradon(ssino_to_recon, 
                                           theta=ds.obincens, 
                                           output_size=outsize,
                                           projection_shifts=np.full(whole_sample_sino.shape, -y0),
                                           filter_name='hamming',
                                           interpolation='linear',
                                           workers=nthreads)
        
        recon_man_mask = apply_manual_mask(recon)
        if manual_threshold is None:
            thresh = threshold_otsu(recon_man_mask)
        else:
            thresh = manual_threshold
            
        binary = recon_man_mask > thresh
        whole_sample_mask = convex_hull_image(binary)
        
        for g in grains:
            g.translation = np.array([g.dx, g.dy, 0])
        
        print("Peak 2D organise")
        pks = ImageD11.sinograms.properties.pks_table.load(ds.pksfile)
        p2d = pks.pk2d(ds.omega, ds.dty)
        numba_order, numba_histo = utils.counting_sort(p2d['spot3d_id'])
        grain_2d_id = utils.palloc(p2d['spot3d_id'].shape, np.dtype(int))
        cleanid = cf_strong.grain_id.copy()
        utils.find_grain_id(cf_strong.spot3d_id, cleanid, p2d['spot3d_id'], grain_2d_id, numba_order)
        gord, counts = utils.counting_sort(grain_2d_id)
        inds = np.concatenate(((0,), np.cumsum(counts)))
        
        for grain in tqdm(grains):
            i = grain.gid
            grain.peaks_2d = gord[inds[i+1] : inds[i+2]]
        
        print("Making sinograms")
        with concurrent.futures.ThreadPoolExecutor(max_workers= max(1,nthreads-1)) as pool:
            for i in tqdm(pool.map(do_sinos, grains), total=len(grains)):
                pass
        
        print("Running iradon")
        
        run_this_iradon = partial(run_iradon_id11, pad=pad, y0=y0, sample_mask=whole_sample_mask, workers=1, apply_halfmask=is_half_scan, mask_central_zingers=is_half_scan)

        with concurrent.futures.ThreadPoolExecutor( max_workers= max(1,nthreads-1) ) as pool:
            for i in tqdm(pool.map(run_this_iradon, grains), total=len(grains)):
                pass
            
        for grain in grains:
            grain.og_recon = grain.recon
            
        save_grains_for_mlem(grains, ds, y0)
        
        bash_script_path, recons_path = prepare_mlem_bash(ds, grains, pad, is_half_scan, mlem_n_simultaneous_jobs, mlem_cores_per_task, mlem_niter)
        
        utils.slurm_submit_and_wait(bash_script_path, 30)
        
        for i, grain in enumerate(tqdm(grains)):
            grain.recon = np.loadtxt(os.path.join(recons_path, ds.dsname + f"_mlem_recon_{i}.txt"))
            
        rgb_x_array, rgb_y_array, rgb_z_array, grain_labels_array, raw_intensity_array = utils.build_slice_arrays(grains, cutoff_level)
        
        save_grains(grains, ds)

print("Done!")